In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit,RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor


In [2]:
from xgboost import XGBRegressor

In [3]:
data1 = pd.read_csv('C:/Users/somya/Downloads/Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis/Data_set/final-data.csv', index_col=0, parse_dates=True)

C:\Users\somya\AppData\Local\Temp\ipykernel_17808\2588901285.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data1 = pd.read_csv('C:/Users/somya/Downloads/Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis/Data_set/final-data.csv', index_col=0, parse_dates=True)


In [4]:
x=data1.drop(['bitcoin_volatility'],axis=1)

In [6]:
y = data1['market_signal']


In [7]:
print(x.isnull().sum())

timestamp                     0
price                         0
total_volume                  0
market_cap                    1
coin_name                     0
bitcoin_returns               0
score                         0
No_Comments                   0
title_sentiment               0
description_sentiment         0
score_lag1                    0
score_lag2                    0
score_lag3                    0
score_lag4                    0
score_lag5                    0
score_lag6                    0
score_lag7                    0
No_Comments_lag1              0
No_Comments_lag2              0
No_Comments_lag3              0
No_Comments_lag4              0
No_Comments_lag5              0
No_Comments_lag6              0
No_Comments_lag7              0
title_sentiment_lag1          0
title_sentiment_lag2          0
title_sentiment_lag3          0
title_sentiment_lag4          0
title_sentiment_lag5          0
title_sentiment_lag6          0
title_sentiment_lag7          0
descript

In [17]:
numeric_cols = x.select_dtypes(include=[np.number]).columns
x[numeric_cols] = x[numeric_cols].fillna(x[numeric_cols].mean())


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# Features
X = data1.drop(columns=['timestamp', 'coin_name', 'market_signal'], errors='ignore')
# Target
y = data1['market_signal']

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Split data (No shuffle for time series!)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, shuffle=False)

# Train classifier
clf_model = RandomForestClassifier(n_estimators=100, random_state=42)
clf_model.fit(X_train, y_train)

# Predict
y_pred = clf_model.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred, target_names=encoder.classes_))


              precision    recall  f1-score   support

     Bearish       0.99      0.86      0.92       226
     Bullish       0.98      0.88      0.93       146
     Neutral       0.89      0.99      0.93       376

    accuracy                           0.93       748
   macro avg       0.95      0.91      0.93       748
weighted avg       0.94      0.93      0.93       748



In [15]:
tscv = TimeSeriesSplit(n_splits=5)

In [24]:
# Make sure to only keep numeric features
X = x.copy()

# Drop non-numeric columns (timestamp, coin_name etc.)
X = X.drop(columns=['timestamp', 'coin_name', 'market_signal'], errors='ignore')


In [28]:
def evaluate_classifier(model, X, y):
    tscv = TimeSeriesSplit(n_splits=5)
    accuracies = []
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        accuracies.append(acc)

    return np.mean(accuracies)


In [29]:
from sklearn.metrics import classification_report, accuracy_score

# Make predictions
y_pred = clf_model.predict(X)

# Print classification report
print(classification_report(y, y_pred, target_names=encoder.classes_))

# Optionally, print accuracy too
print(f"Accuracy: {accuracy_score(y, y_pred)}")


              precision    recall  f1-score   support

     Bearish       0.00      0.00      0.00       260
     Bullish       0.85      1.00      0.92      1709
     Neutral       0.86      0.84      0.85      1770

    accuracy                           0.85      3739
   macro avg       0.57      0.61      0.59      3739
weighted avg       0.79      0.85      0.82      3739

Accuracy: 0.8529018454132121


c:\Users\somya\Downloads\Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis\cryptoenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\somya\Downloads\Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis\cryptoenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\somya\Downloads\Hybrid Model for Crypto Volatility Prediction using Sentiment Analysis\cryptoenv\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision

In [30]:
clf_accuracy = evaluate_classifier(clf_model, X, y)
print(f"Random Forest Classifier - Accuracy: {clf_accuracy}")



Random Forest Classifier - Accuracy: 0.9756019261637239


In [32]:
import joblib

# Save model
joblib.dump(clf_accuracy, 'random_forest_market_signal_model.joblib')


['random_forest_market_signal_model.joblib']

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_estimator(clf, X_test, y_test, display_labels=encoder.classes_, cmap="Blues")
plt.title("Confusion Matrix - Market Signal Prediction")
plt.show()


In [ ]:
import pandas as pd

feature_importances = pd.Series(clf.feature_importances_, index=X.columns)
feature_importances = feature_importances.sort_values(ascending=False)

print(feature_importances.head(10)) 

(366, 47)

(362, 47)

In [ ]:
sen=sum(se)/len(se)

In [ ]:
sen

-0.54035959875

In [ ]:
se

np.float64(-0.5403595987500001)

In [ ]:
se.mean()

AttributeError: 'list' object has no attribute 'mean'

In [ ]:
df=df.rename(columns={'Unnamed: 0':'timestamp'})

In [ ]:
df.set_index('timestamp', inplace=True)

In [ ]:
df.shape

(16, 46)

In [ ]:
df.head()

,bitcoin_price,bitcoin_volume,bitcoin_market_cap,bitcoin_returns,bitcoin_volatility,score,No_Comments,title_sentiment,description_sentiment,score_lag1,...,description_sentiment_lag7,sentiment,sentiment1,sentiment2,sentiment3,sentiment4,sentiment5,sentiment6,sentiment7,total_sentiment
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-24,63327.026545,2.399885e+10,1.251095e+12,-0.000882,0.397836,7.000000,11.0,-0.966808,-0.249912,29.667235,...,-0.066157,-0.608360,-0.225495,-0.231196,-0.242698,-0.25057,-0.248735,-0.247549,-0.278639,-0.291655
2024-09-25,64334.142749,3.149098e+10,1.271184e+12,0.015903,0.397836,29.273970,114.0,-0.417772,-0.023243,7.000000,...,-0.066157,-0.220508,-0.608360,-0.231196,-0.242698,-0.25057,-0.248735,-0.247549,-0.278639,-0.291032
2024-09-26,63151.899594,2.608764e+10,1.245721e+12,-0.018377,0.397836,42.666667,16.0,-0.321892,-0.500518,29.273970,...,-0.066157,-0.411205,-0.220508,-0.608360,-0.242698,-0.25057,-0.248735,-0.247549,-0.278639,-0.313533
2024-09-27,65130.768840,3.799557e+10,1.287151e+12,0.031335,0.397836,2.181818,4.0,-0.648641,0.362800,42.666667,...,-0.066157,-0.142921,-0.411205,-0.220508,-0.608360,-0.25057,-0.248735,-0.247549,-0.278639,-0.301061
2024-09-28,65791.002125,3.266492e+10,1.299708e+12,0.010137,0.397836,2.571429,22.0,-0.441387,-0.069390,2.181818,...,-0.066157,-0.255388,-0.142921,-0.411205,-0.220508,-0.60836,-0.248735,-0.247549,-0.278639,-0.301663
